<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Kevin Pinochet Hernández
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/Keviinplz/MDS7202-1`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Introducción a procesamiento del lenguaje natural.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 26/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Visualizar el funcionamiento de clasificadores.
- Realizar una GridSearch sobre un conjunto de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [2]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Librería para NLP
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kevii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [3]:
import os

path: str
def get_filepath_by_name(filename: str) -> str:
    return os.path.join(os.path.join(os.getcwd(), 'data'), filename)

In [4]:
df_comics = pd.read_csv(get_filepath_by_name('df_comics.csv'))
df_comics_no_label = pd.read_csv(get_filepath_by_name('comics_no_label.csv'))
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

def exploratory_data_analysis(df: pd.DataFrame) -> None:
    """ Realiza una exploración del dataframe según lo pedido"""
    
    print(f"1.- El DataFrame tiene {len(df)} filas y {len(df.columns)} columnas\n")
    print(f"2.- El DataFrame está compuesto por las siguientes columnas: {list(df.columns)}\n")
    print("3.- Ejemplos de filas del DataFrame:\n\nPrimeras 5 filas:\n")
    display(df.head(5))
    print("Ultimas 5 filas")
    display(df.tail(5))
    print("Muestreo aleatorio de 5 filas:")
    display(df.sample(5))
    print("4.- Descripción numérica del Dataframe:")
    display(df.describe())
    print("5.- Cantidad de valores nulos por columna:")
    display(df.isnull().sum())
    print("6.- Cantidad de valores únicos por columna:")
    display(df.nunique())
    duplicates: int = df.duplicated(subset=['name']).sum()
    print(f"7.- Presenta un total de {duplicates} filas con nombres duplicados.\n")
    if duplicates > 0:
        print("8.- Las filas con nombres duplicados son:")
        display(df[df.duplicated(subset=['name'])])
    #df.plot.box()

exploratory_data_analysis(df_comics)


1.- El DataFrame tiene 1285 filas y 82 columnas

2.- El DataFrame está compuesto por las siguientes columnas: ['Unnamed: 0', 'name', 'real_name', 'full_name', 'overall_score', 'history_text', 'powers_text', 'intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score', 'superpowers', 'alter_egos', 'aliases', 'place_of_birth', 'first_appearance', 'creator', 'alignment', 'occupation', 'base', 'teams', 'relatives', 'gender', 'type_race', 'height', 'weight', 'eye_color', 'hair_color', 'skin_color', 'img', 'has_electrokinesis', 'has_energy_constructs', 'has_mind_control_resistance', 'has_matter_manipulation', 'has_telepathy_resistance', 'has_mind_control', 'has_enhanced_hearing', 'has_dimensional_travel', 'has_element_control', 'has_size_changing', 'has_fire_resistance', 'has_fire_control', 'has_dexterity', 'has_reality_warping', 'has_illusions', 'has_energy_beams', 'has_peak_human_condition', 'has_shapeshifting', 'has_heat_resistance', 'has_jump',

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,40,70,"['Super Speed', 'Super Strength']",[],[''],NaN,NaN,Marvel Comics,Good,NaN,NaN,"['Annihilators', 'Asgardians', 'Avengers', 'Ne...",NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/10038.jpg?v=156096...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,100,80,"['Accelerated Healing', 'Agility', 'Berserk Mo...",[],['Rick Jones'],"Scarsdale, Arizona","Hulk Vol 2 #2 (April, 2008) (as A-Bomb)",Marvel Comics,Good,"Musician, adventurer, author; formerly talk sh...",NaN,"['Teen Brigade', 'Ultimate Fantastic Four', 'U...",Marlo Chandler-Jones (wife); Polly (aunt); Mrs...,Male,Human,6'8 • 203 cm,980 lb • 441 kg,Yellow,No Hair,NaN,/pictures2/portraits/10/050/10060.jpg?v=158233...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,100,55,"['Energy Absorption', 'Energy Armor', 'Energy ...",[],[''],Stoneworld,Green Lantern Vol 3 #21,DC Comics,Good,NaN,NaN,"['Blue Lantern Corps', 'Green Lantern Corps', ...",NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/10/050/1410.jpg?v=1581168103,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,30,50,"['Weapon-based Powers', 'Weapons Master']",[],[''],Gotham City,NaN,DC Comics,Good,NaN,NaN,[],NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/11650.jpg?v=156173...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,55,85,"['Accelerated Healing', 'Agility', 'Astral Pro...",[],[''],NaN,NaN,George Lucas,Good,NaN,NaN,['Jedi Order'],NaN,Female,Twi'lek,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/10891.jpg?v=156181...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

Ultimas 5 filas


,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
1362,1445,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,100,55,"['Cryokinesis', 'Fire Control', 'Magic', 'Prob...","['Chaos Zatanna', 'Doctor Fate', 'Zatanna (Ful...",[''],NaN,Hawkman #4,DC Comics,Good,NaN,NaN,"['Seven Soldiers of Victory', 'Sentinels of Ma...","Giovanni ""John"" Zatara (father, deceased), Sin...",Female,Human,5'7 • 170 cm,127 lb • 57 kg,Blue,Black,NaN,/pictures2/portraits/10/050/809.jpg?v=1578913192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1363,1446,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,NaN,80,100,100,100,100,80,"['Accelerated Healing', 'Acrobatics', 'Agility...",[],[''],NaN,Mega Man X (1993),Capcom,Good,NaN,NaN,[],NaN,Male,Robot,5'6 • 168 cm,145 lb • 65 kg,Blue,Blond,Red,/pictures2/portraits/10/050/12699.jpg?v=158402...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1364,1447,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,100,80,"['Accelerated Healing', 'Agility', 'Durability...","['Black Flash (CW)', 'Zoom', 'Zoom (CW)']",['Judge · Reverse-Flash · The Flash'],NaN,Flash Secret Files and Origins #3,DC Comics,Bad,"Criminal · former F.B.I. Profiler, Politician","Keystone City, Kansas",['Flash Family'],Ashley Zolomon (ex-wife),Male,NaN,6'1 • 185 cm,181 lb • 81 kg,Red,Brown,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1365,1448,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,100,30,"['Intangibility', 'Super Speed', 'Time Manipul...","['Black Flash (CW)', 'Zoom (CW)', 'Zoom (New 5...",[''],NaN,Flash Secret Files #3,DC Comics,Bad,NaN,"Keystone City, Kansas","['Rogues', 'The Society', 'Flash Family', 'Jus...",Ashley Zolomon (ex-wife),Male,NaN,6'1 • 185 cm,181 lb • 81 kg,Red,Brown,NaN,/pictures2/portraits/10/050/1384.jpg?v=1295759009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1366,1449,Zzzax,NaN,NaN,5,Zzzax was th

Muestreo aleatorio de 5 filas:


...

5.- Cantidad de valores nulos por columna:


Unnamed: 0                             0
name                                   0
real_name                            121
full_name                            390
overall_score                          0
history_text                           0
powers_text                          324
intelligence_score                     0
strength_score                         0
speed_score                            0
durability_score                       0
power_score                            0
combat_score                           0
superpowers                            0
alter_egos                             0
aliases                                0
place_of_birth                       551
first_appearance                     143
creator                              119
alignment                              0
occupation                           347
base                                 472
teams                                  0
relatives                            456
gender          

6.- Cantidad de valores únicos por columna:


Unnamed: 0                          1285
name                                1285
real_name                            971
full_name                            794
overall_score                         87
history_text                        1274
powers_text                          939
intelligence_score                    16
strength_score                        21
speed_score                           21
durability_score                      21
power_score                           21
combat_score                          21
superpowers                         1166
alter_egos                           463
aliases                              860
place_of_birth                       422
first_appearance                    1003
creator                               39
alignment                              3
occupation                           753
base                                 519
teams                                390
relatives                            775
gender          

7.- Presenta un total de 0 filas con nombres duplicados.



In [7]:
from typing import List

# Lo verdaderamente importante es la columna name, que es la que se usa para identificar los comics
# la columna history_text, que es la que se usa para analizar el texto de la historia
# y los atributos numéricos
# Por lo que se limpiará el dataset de nulls y duplicados de la columna name, nulls en la columna history_text y nulls en los atributos numéricos
numeric_fields: List[str] = list(df_comics.select_dtypes(include=['int64', 'float64']).columns)
df = df_comics.dropna(subset=['name', 'history_text'] + numeric_fields)
display(df.isnull().sum())
print(f"El DataFrame ahora tiene {len(df)} filas y {len(df.columns)} columnas\n")

Unnamed: 0                             0
name                                   0
real_name                            113
full_name                            378
overall_score                          0
history_text                           0
powers_text                          301
intelligence_score                     0
strength_score                         0
speed_score                            0
durability_score                       0
power_score                            0
combat_score                           0
superpowers                            0
alter_egos                             0
aliases                                0
place_of_birth                       530
first_appearance                     129
creator                              118
alignment                              0
occupation                           330
base                                 451
teams                                  0
relatives                            435
gender          

El DataFrame ahora tiene 1228 filas y 82 columnas



In [8]:
display(df.isnull().sum())

Unnamed: 0                             0
name                                   0
real_name                            113
full_name                            378
overall_score                          0
history_text                           0
powers_text                          301
intelligence_score                     0
strength_score                         0
speed_score                            0
durability_score                       0
power_score                            0
combat_score                           0
superpowers                            0
alter_egos                             0
aliases                                0
place_of_birth                       530
first_appearance                     129
creator                              118
alignment                              0
occupation                           330
base                                 451
teams                                  0
relatives                            435
gender          

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `bag of words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `bag of words`?...

`bag of words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `bag of words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `bag of words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [9]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [10]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [11]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [12]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
_df = bow.fit_transform(docs)

pd.DataFrame(_df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [13]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
_df = bow.fit_transform(docs)

pd.DataFrame(_df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,good rock,like,like good,new,new york,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar MinMaxScaler sobre los atributos de interes.

**Respuesta:**

In [14]:
#### Código aquí ####
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

pipe: Pipeline = Pipeline([
    ("bow-minmax", ColumnTransformer(transformers=[
        ("bow", CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2)), "history_text"),
        ("minmax", MinMaxScaler(), atributos_de_interes),
    ]))
])

transformed: pd.DataFrame = pipe.fit_transform(df)
transformed

<1228x327651 sparse matrix of type '<class 'numpy.float64'>'
	with 773416 stored elements in Compressed Sparse Row format>

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**To-DO:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1 y aplicar un clasificador  `MultinomialNB()`.
- [ ] Entrenar el pipeline y comentar los resultados.

**Respuesta:**

In [15]:
#### Código aquí ####
seed: int = 42
pipe_multinomial: Pipeline = Pipeline([
    ("bow-minmax", ColumnTransformer(transformers=[
        ("bow", CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2)), "history_text"),
        ("minmax", MinMaxScaler(), atributos_de_interes),
    ])),
    ("features-selector", SelectPercentile(f_classif, percentile=90)),
    ("multinomial-classifier", MultinomialNB())
])

X, y = df.loc[:, df.columns != "alignment"], df.loc[:, df.columns == "alignment"].to_numpy().ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
transformed: np.ndarray = pipe_multinomial.fit(X_train, y_train)

In [16]:
print(classification_report(y_test, transformed.predict(X_test)))

              precision    recall  f1-score   support

         Bad       0.59      0.09      0.16       112
        Good       0.61      0.98      0.75       219
     Neutral       1.00      0.03      0.05        38

    accuracy                           0.61       369
   macro avg       0.73      0.37      0.32       369
weighted avg       0.65      0.61      0.50       369



In [17]:
df["alignment"].value_counts()

Good       702
Bad        414
Neutral    112
Name: alignment, dtype: int64

In [18]:
# Podemos observar que clasifica bastante bien la clase Neutral, pero no la clase Bad o Good
# Sin embargo, su recall es muy malo (al igual que Bad), lo que quiere decir, que en realidad hay muchos falsos positivos
# En las clasificaciones (predice como Bad o Neutral muchas etiquetas que en realidad son Good)
# Esto es debido a que Good contiene muchísimos más ejemplos que las demás clases, por lo que el modelo
# Aprendió a clasificar bien Good, pero no a clasificar bien Bad o Neutral

# Se concluye que el modelo no es capaz de clasificar bien.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Dos clasificadores distintos en donde varie sus parámetros. Se le recomienda utilizar `LogisticRegression()` y `RandomForestClassifier()`.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. 
- Selección de las mejores columnas para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
        'clf__multi_class': ['liblinear']},
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [23]:
#### Código aquí ####
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV


logistic_params = [
    {'features-selector__percentile': [20, 40, 60, 80]},
    {'bow-minmax__bow__ngram_range': [(1,1), (1,2), (1,3)]},
]

random_forest_params = [
    {'features-selector__percentile': [20, 40, 60, 80]},
    {'bow-minmax__bow__ngram_range': [(1,1), (1,2), (1,3)]},
]

pipe_logistic: Pipeline = Pipeline([
    ("bow-minmax", ColumnTransformer(transformers=[
        ("bow", CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2)), "history_text"),
        ("minmax", MinMaxScaler(), atributos_de_interes),
    ])),
    ("features-selector", SelectPercentile(f_classif, percentile=90)),
    ("logistic", LogisticRegression())
])

pipe_random_forest: Pipeline = Pipeline([
    ("bow-minmax", ColumnTransformer(transformers=[
        ("bow", CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2)), "history_text"),
        ("minmax", MinMaxScaler(), atributos_de_interes),
    ])),
    ("features-selector", SelectPercentile(f_classif, percentile=90)),
    ("random-forest", RandomForestClassifier())
])


In [24]:
search_logistic = HalvingGridSearchCV(pipe_logistic, logistic_params, verbose=10)
search_random_forest = HalvingGridSearchCV(pipe_random_forest, random_forest_params, verbose=10)

In [25]:
search_logistic.fit(X_train, y_train)
search_random_forest.fit(X_train, y_train)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 286
max_resources_: 859
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 7
n_resources: 286
Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5; 1/7] START features-selector__percentile=20............................


C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab8--M51uw4k-py3.9\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5; 1/7] END features-selector__percentile=20;, score=(train=1.000, test=0.474) total time=   5.0s
[CV 2/5; 1/7] START features-selector__percentile=20............................


C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab8--M51uw4k-py3.9\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5; 1/7] END features-selector__percentile=20;, score=(train=1.000, test=0.579) total time=   6.2s
[CV 3/5; 1/7] START features-selector__percentile=20............................


...

C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab8--M51uw4k-py3.9\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5; 3/3] END bow-minmax__bow__ngram_range=(1, 1);, score=(train=0.993, test=0.700) total time=  11.0s


C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab8--M51uw4k-py3.9\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 286
max_resources_: 859
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 7
n_resources: 286
Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5; 1/7] START features-selector__percentile=20............................
[CV 1/5; 1/7] END features-selector__percentile=20;, score=(train=1.000, test=0.526) total time=   4.1s
[CV 2/5; 1/7] START features-selector__percentile=20............................
[CV 2/5; 1/7] END features-selector__percentile=20;, score=(train=1.000, test=0.544) total time=   3.9s
[CV 3/5; 1/7] START features-selector__percentile=20............................
[CV 3/5; 1/7] END features-selector__percentile=20;, score=(train=1.000, test=0.509) total time=   3.8s
[CV 4/5; 1/7] START features-selector__percentile=20............................
[CV 4/5; 1/7] END features-selector__percentile=20;, score=(train=1.000, test=0.561) total time=   4.1s
[C

HalvingGridSearchCV(estimator=Pipeline(steps=[('bow-minmax',
                                               ColumnTransformer(transformers=[('bow',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x0000023940FCBD60>),
                                                                                'history_text'),
                                                                               ('minmax',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                            

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [29]:
#### Código aquí ####
df_comics_no_label = pd.read_csv(get_filepath_by_name("comics_no_label.csv"))
df_comics_no_label.head()

,Unnamed: 0,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,alignment,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,0.0,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.0,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,613.369048,82.321429,40.476190,47.380952,57.619048,64.523810,69.226190,NaN,0.102564,0.038462,0.076923,0.051282,0.089744,0.051282,0.064103,0.012821,0.192308,0.064103,0.141026,0.115385,0.064103,0.0,0.025641,0.051282,0.128205,0.038462,0.217949,0.089744,0.076923,0.102564,0.205128,0.115385,0.038462,0.192308,0.076923,0.166667,0.115385,0.089744,0.089744,0.115385,0.141026,0.115385,0.294872,0.153846,0.153846,0.230769,0.166667,0.256410,0.179487,0.230769,0.448718,0.320513,0.269231,0.243590,0.500000,0.346154,0.397436,0.500000
std,463.554958,23.624146,31.168209,25.239530,28.469770,33.470172,27.935964,NaN,0.305352,0.193552,0.268194,0.222000,0.287664,0.222000,0.246521,0.113228,0.396664,0.246521,0.350301,0.321553,0.246521,0.0,0.159085,0.222000,0.336482,0.193552,0.415525,0.287664,0.268194,0.305352,0.406409,0.321553,0.193552,0.396664,0.268194,0.375090,0.321553,0.287664,0.287664,0.321553,0.350301,0.321553,0.458936,0.363137,0.363137,0.424052,0.375090,0.439477,0.386244,0.424052,0.500582,0.469694,0.446431,0.432026,0.503236,0.478822,0.492535,0.503236
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,193.250000,80.000000,13.750000,30.000000,40.000000,40.000000,60.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,537.500000,90.000000,30.000000,50.000000,60.000000,70.000000,75.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [30]:
exploratory_data_analysis(df_comics_no_label)

1.- El DataFrame tiene 84 filas y 82 columnas

2.- El DataFrame está compuesto por las siguientes columnas: ['Unnamed: 0', 'name', 'real_name', 'full_name', 'overall_score', 'history_text', 'powers_text', 'intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score', 'superpowers', 'alter_egos', 'aliases', 'place_of_birth', 'first_appearance', 'creator', 'alignment', 'occupation', 'base', 'teams', 'relatives', 'gender', 'type_race', 'height', 'weight', 'eye_color', 'hair_color', 'skin_color', 'img', 'has_electrokinesis', 'has_energy_constructs', 'has_mind_control_resistance', 'has_matter_manipulation', 'has_telepathy_resistance', 'has_mind_control', 'has_enhanced_hearing', 'has_dimensional_travel', 'has_element_control', 'has_size_changing', 'has_fire_resistance', 'has_fire_control', 'has_dexterity', 'has_reality_warping', 'has_illusions', 'has_energy_beams', 'has_peak_human_condition', 'has_shapeshifting', 'has_heat_resistance', 'has_jump', '

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,35,100,"['Durability', 'Reflexes', 'Super Strength']","['Batgod', 'Batman', 'Batman (1966)', 'Batman ...","['Subject 514A', 'Bruce Wayne', 'Bruce 2']",NaN,NaN,DC Comics,NaN,NaN,NaN,[],Bruce Wayne (genetic template),NaN,NaN,-,-,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,55,80,90,"['Agility', 'Banish', 'Energy Blasts', 'Enhanc...",[],[''],NaN,NaN,DC Comics,NaN,NaN,NaN,['Challengers of the Unknown'],NaN,NaN,NaN,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/11545.jpg?v=156107...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,55,65,100,"['Element Control', 'Heat Generation', 'Light ...",[],[''],"Terrace of Light, Shattrath City",NaN,Blizzard Entertainment,NaN,NaN,NaN,[],NaN,NaN,NaN,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/14754.jpg?v=156121...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,25,35,80,"['Absorption', 'Agility', 'Energy Absorption',...",[],"['Maverick', 'Zero']",NaN,NaN,Marvel Comics,NaN,NaN,NaN,[],NaN,NaN,Mutant,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/14359.jpg?v=155405...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,75,45,65,"['Dexterity', 'Durability', 'Intelligence', 'I...",['Ajax'],"['A British Villain', 'British Shitstick', 'Co...",NaN,NaN,Marvel Comics,NaN,NaN,NaN,[],NaN,Male,Mutant,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/14265.jpg?v=155393...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


Ultimas 5 filas


,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
79,1394,Weapon Hex,Laura Kinney,NaN,-,NaN,NONE NOTABLE,0,0,0,0,0,0,[],"['Captain Universe (X-23)', 'Old Woman Laura',...",['Hex-23'],Transia,Infinity Wars #3,Marvel Comics,NaN,Adventurer,Citadel of the Defendersr,[],"Sarah Kinney (mother, deceased), Herbert Wyndh...",Female,Mutant,-,-,Blue,Red,NaN,/pictures2/portraits/10/050/15161.jpg?v=157942...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1396,Wendigo,Wendigo,NaN,6,The Wendigo are the result of an ancient curse...,NaN,65,80,75,75,90,70,"['Cold Resistance', 'Fire Resistance', 'Heat R...",[],[''],Canada,Incredible Hulk #162,Marvel Comics,NaN,NaN,Canada,[],NaN,NaN,NaN,-,-,Green,White,NaN,/pictures2/portraits/10/050/11256.jpg?v=155534...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81,1431,XS (CW),Nora West-Allen,NaN,19,"According to Eobard Thawne, the daughter of Ba...",Nora possesses a natural connection to the Spe...,90,55,100,60,100,65,"['Accelerated Healing', 'Agility', 'Bullet Tim...",[],[''],NaN,NaN,NaN,NaN,Forensic scientist at the Central City Police ...,NaN,[],Barry Allen (father)\nIris West-Allen (mother)...,NaN,Metahuman,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/15939.jpg?v=155568...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
82,1440,Yondu (MCU),Yondu,NaN,7,"Yondu Udonta was a Centaurian, leader of a key...","Yondu Udonta was a member of the Centaurians, ...",85,25,40,40,60,80,"['Marksmanship', 'Weapon-based Powers', 'Weapo...",[],['Pirate'],NaN,Guardians of the Galaxy,Marvel Comics,NaN,NaN,NaN,[],NaN,Male,NaN,5'6 • 168 cm,130 lb • 59 kg,Red,Red,NaN,/pictures2/portraits/11/050/13901.jpg?v=155387...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,10,20,"['Animal Attributes', 'Animal Oriented Powers']","['Milkman Man', 'Red Torpedo', 'Red Volcano']","[""Battlin' Bovine""]",NaN,NaN,DC Comics,NaN,NaN,Wayne Manor,[],NaN,NaN,Animal,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/13425.jpg?v=157425...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Muestreo aleatorio de 5 filas:


...

Unnamed: 0                           0
name                                 2
real_name                           18
full_name                           71
overall_score                        0
history_text                         8
powers_text                         41
intelligence_score                   0
strength_score                       0
speed_score                          0
durability_score                     0
power_score                          0
combat_score                         0
superpowers                          0
alter_egos                           0
aliases                              0
place_of_birth                      64
first_appearance                    46
creator                             10
alignment                           84
occupation                          53
base                                58
teams                                0
relatives                           58
gender                              45
type_race                

6.- Cantidad de valores únicos por columna:


Unnamed: 0                          83
name                                81
real_name                           63
full_name                           12
overall_score                       24
history_text                        75
powers_text                         43
intelligence_score                  11
strength_score                      18
speed_score                         19
durability_score                    20
power_score                         21
combat_score                        16
superpowers                         77
alter_egos                          30
aliases                             32
place_of_birth                      19
first_appearance                    37
creator                             10
alignment                            0
occupation                          31
base                                25
teams                               10
relatives                           26
gender                               2
type_race                

7.- Presenta un total de 2 filas con nombres duplicados.

8.- Las filas con nombres duplicados son:


,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
32,398,NaN,NaN,NaN,-,NaN,None known except that he may possess the abil...,0,0,0,0,0,0,[],[],[''],NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/pictures2/portraits/10/050/1018.jpg?v=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,10,20,"['Animal Attributes', 'Animal Oriented Powers']","['Milkman Man', 'Red Torpedo', 'Red Volcano']","[""Battlin' Bovine""]",NaN,NaN,DC Comics,NaN,NaN,Wayne Manor,[],NaN,NaN,Animal,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/13425.jpg?v=157425...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) # eliminar ejemplos sin historia
df_comics_no_label = df_comics_no_label.drop_duplicates(subset=['history_text']) # eliminar ejemplos duplicados


In [43]:
df_comics_no_label.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,35,100,"['Durability', 'Reflexes', 'Super Strength']","['Batgod', 'Batman', 'Batman (1966)', 'Batman ...","['Subject 514A', 'Bruce Wayne', 'Bruce 2']",NaN,NaN,DC Comics,NaN,NaN,NaN,[],Bruce Wayne (genetic template),NaN,NaN,-,-,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,55,65,100,"['Element Control', 'Heat Generation', 'Light ...",[],[''],"Terrace of Light, Shattrath City",NaN,Blizzard Entertainment,NaN,NaN,NaN,[],NaN,NaN,NaN,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/14754.jpg?v=156121...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,25,35,80,"['Absorption', 'Agility', 'Energy Absorption',...",[],"['Maverick', 'Zero']",NaN,NaN,Marvel Comics,NaN,NaN,NaN,[],NaN,NaN,Mutant,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/14359.jpg?v=155405...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,75,45,65,"['Dexterity', 'Durability', 'Intelligence', 'I...",['Ajax'],"['A British Villain', 'British Shitstick', 'Co...",NaN,NaN,Marvel Comics,NaN,NaN,NaN,[],NaN,Male,Mutant,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/14265.jpg?v=155393...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
6,46,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,100,100,100,"['Afterimage Creation', 'Cold Resistance', 'Cr...",[],[''],NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/15565.jpg?v=156172...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
to_search: pd.DataFrame = df_comics_no_label[df_comics_no_label["name"].isin(["Vergil", "Gorilla Girl", "Batcow"])]

X_s, y_s = to_search.loc[:, to_search.columns != "alignment"], to_search.loc[:, to_search.columns == "alignment"].to_numpy().ravel()

In [55]:
X_s

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,10,20,"['Animal Attributes', 'Animal Oriented Powers']","['Milkman Man', 'Red Torpedo', 'Red Volcano']","[""Battlin' Bovine""]",NaN,NaN,DC Comics,NaN,Wayne Manor,[],NaN,NaN,Animal,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/13425.jpg?v=157425...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,60,45,100,"['Agility', 'Jump', 'Super Strength', 'Transfo...",[],['Gorilla Woman'],NaN,NaN,Marvel Comics,NaN,NaN,[],NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,90,100,100,"['Accelerated Healing', 'Agility', 'Duplicatio...",[],"['Gilver, Nelo Angelo, Son of Sparda, The Dark...",NaN,Devil May Cry,Capcom,NaN,NaN,[],"Sparda (father) ,Eva (mother) ,Dante (Brother)...",Male,NaN,6'3 • 191 cm,207 lb • 93 kg,Blue,White,NaN,/pictures2/portraits/10/050/11657.jpg?v=154990...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [57]:
y_logistic = search_logistic.predict(X_s)
print(f""" 
Batcow: {y_logistic[0]}
Gorilla Girl: {y_logistic[1]}
Vergil: {y_logistic[2]}
""")

 
Batcow: Good
Gorilla Girl: Good
Vergil: Good



In [58]:
y_random_forest = search_random_forest.predict(X_s)
print(f""" 
Batcow: {y_random_forest[0]}
Gorilla Girl: {y_random_forest[1]}
Vergil: {y_random_forest[2]}
""")

 
Batcow: Good
Gorilla Girl: Good
Vergil: Good



# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>